In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random
import scipy as sp
# Import TensorFlow and NumPy
# 초기 변수들 지정
import tensorflow as tf
import numpy as np
import scipy as sp


In [3]:
# 단면적 계산(지배방정식)
def cal_area(center_a,center_b,d,theta_1,theta_2,R):

    theta = relu(theta_1 - theta_2)

    if theta == 0 :
        return 0

    point_1 = center_a +(center_b*center_a)/(R*tf.sin(theta_1))

    point_2 = center_a +(center_b*(center_a-d))/(R*tf.sin(theta_2))

    fan_shape = (1/2) * theta * (pow(R,2))

    triangle_1 = (1/2)  * (0-point_1) * (R*tf.sin(theta_1)+center_b)

    triangle_2 = (1/2)  * (d - point_2) * (R*tf.sin(theta_2)+center_b)

    triangle_3 = (1/2)  * (point_2 - point_1) * (0-center_b)

    area = fan_shape - triangle_1 + triangle_2 - triangle_3

    # print(f'theta_1: {math.degrees(theta_1)} theta_2: {math.degrees(theta_2)} theta: {math.degrees(theta)}')

    return area

def relu(theta):

    return np.maximum(0,theta)


In [5]:
# Import TensorFlow and NumPy
# 초기 변수들 지정
import tensorflow as tf
import numpy as np
import scipy as sp

# Set data type
DTYPE='float64'
tf.keras.backend.set_floatx(DTYPE)

# Define contact line velocity
# theta_0 : equilibrium contact angle
# theta_d : dynamic contact angle
def fun_contact_line_velocity( f_c,i_t,theta_0, theta_d ):
    one = tf.constant(1, dtype=tf.float64)
    return -f_c*i_t*(-tf.cos(theta_0)+tf.cos(theta_d))
# Set constants
pi = tf.constant(np.pi, dtype=tf.float64)
p_f_c = tf.constant(-2.02, dtype=tf.float64) # positive_friction_coefficient
n_f_c = tf.constant(-5.64, dtype=tf.float64) # nagative_friction_coefficient
i_t = tf.constant(7.6408/1000, dtype=tf.float64) #interfacial tension between the two liquids
d = tf.constant(0.321/1000, dtype=tf.float64) #gab

thetal_0 = tf.constant(np.deg2rad(94), dtype=tf.float64)
thetal_d = tf.constant(np.deg2rad(55), dtype=tf.float64)
thetar_0 = tf.constant(np.deg2rad(59),dtype=tf.float64)
thetar_d = tf.constant(np.deg2rad(109),dtype=tf.float64)

# 반지름
r_0 = tf.constant(np.deg2rad(109), dtype=tf.float64)
# 원의 중심 x좌표
cneter_a = tf.constant(-0.0000566758, dtype=tf.float64)
# 원의 중심 y좌표
cneter_b = tf.constant(-0.0006874250504951296, dtype=tf.float64)

#  세타1 좌표
theta1_0 = tf.constant(np.pi-np.deg2rad(94),dtype=tf.float64)
#  세타2 좌표
theta2_0 =  thetar_0

yl_0 = tf.constant(r_0*tf.sin(theta1_0)+cneter_b,shape=(1,1) , dtype=tf.float64)
yr_0 = tf.constant(r_0*tf.sin(theta1_0)+cneter_b,shape=(1,1) , dtype=tf.float64)

# bundary condio
vl = tf.constant(-fun_contact_line_velocity(n_f_c ,i_t,thetal_0, thetal_d ),shape=(1,1) , dtype=tf.float64)
vr = tf.constant(-fun_contact_line_velocity(p_f_c ,i_t,thetar_0 , thetar_d ),shape=(1,1) , dtype=tf.float64)

# initial_data = [a0 ,b0,r0 ,theta_l0,theta_r0 ]
# cal_area_theta(center_a,center_b,d,theta_l,theta_r,R):
#initial_area = tf.constant(5.375716695978319e-08,shape=(1,1) , dtype=tf.float64)
#print(initial_area)
#thetar_d = tf.constant(initial_area, dtype=tf.float64)
# (yl_t-vl) + (yr_t-vr)
# Define residual of the condition
def fun_r(area_t ):
    return (area_t)



In [6]:
# Set random seed for reproducible results
tf.random.set_seed(0)
t_data = tf.random.uniform(shape=[1,10000],minval=0.0,maxval=0.0280, dtype=DTYPE)
t_data = tf.reshape(t_data,shape=(-1,1))
t_data = tf.sort(t_data,axis=0)
print(t_data)

tf.Tensor(
[[1.27272500e-06]
 [8.12988319e-06]
 [1.36279060e-05]
 ...
 [2.79928773e-02]
 [2.79933925e-02]
 [2.79937181e-02]], shape=(10000, 1), dtype=float64)


In [10]:
# Define model architecture
class PINN_NeuralNet(tf.keras.Model):
    """ Set basic architecture of the PINN model."""

    def __init__(self, t_data,
            output_dim=5,
            num_hidden_layers=8,
            num_neurons_per_layer=20,
            activation='tanh',
            kernel_initializer='glorot_normal',
            **kwargs):

        super().__init__(**kwargs)

        self.num_hidden_layers = num_hidden_layers
        self.output_dim = output_dim
        self.t_data = t_data

        # Define NN architecture
        #self.scale = tf.keras.layers.Lambda(
        #    lambda x: 2.0*(x - lb)/(ub - lb) - 1.0)
        self.hidden = [tf.keras.layers.Dense(num_neurons_per_layer,
                             activation=tf.keras.activations.get(activation),
                             kernel_initializer=kernel_initializer)
                           for _ in range(self.num_hidden_layers)]
        self.out = tf.keras.layers.Dense(output_dim)

    def call(self, X):
        """Forward-pass through neural network."""
        for i in range(self.num_hidden_layers):
            X = self.hidden[i](X)
        return self.out(X)

In [8]:
import scipy.optimize

class PINNSolver():
    def __init__(self,model,t_data):
        self.model = model
        # Store collocation points
        self.t_data = t_data
        # Initialize history of losses and global iteration counter
        self.hist = []
        self.iter = 0

    def get_r(self):

        t = self.t_data
        area = []

        for i in range(len(t_data[0])) :
            u_i = self.model(t_data[0][i])
            a = tf.reshape(u_i[0][0],shape=(-1,1))
            b = tf.reshape(u_i[0][1],shape=(-1,1))
            R = tf.reshape(u_i[0][2],shape=(-1,1))
            theta_l =tf.reshape(u_i[0][3],shape=(-1,1))
            theta_r = tf.reshape(u_i[0][4],shape=(-1,1))
            area.append(cal_area(a,b,d,theta_l,theta_r,R))

        # A tf.GradientTape is used to compute derivatives in TensorFlow
        with tf.GradientTape(persistent=True) as tape:
            # Split t and x to compute partial derivatives

            # Variables t and x are watched during tape
            tape.watch(t)
            area = tf.reshape(tf.Variable(area),(1,-1))

            # Determine residual

        area_t = tape.gradient(area, t)

        del tape

        return area_t

    def get_boundary(self):

        # A tf.GradientTape is used to compute derivatives in TensorFlow
        with tf.GradientTape(persistent=True) as tape:
            # Split t and x to compute partial derivatives
            t = self.t_data

            # Variables t and x are watched during tape
            tape.watch(t)

            # Determine residual
            u = self.model(t_data)
            a = tf.reshape(u[:,0],shape=(-1,1))
            b = tf.reshape(u[:,1],shape=(-1,1))
            R = tf.reshape(u[:,2],shape=(-1,1))
            theta_l =tf.reshape(u[:,3],shape=(-1,1))
            theta_r = tf.reshape(u[:,4],shape=(-1,1))

            yl = R*tf.sin(theta_l)+b
            yr = R*tf.sin(theta_r)+b

        yl_t = tape.gradient(yl, t)
        yr_t = tape.gradient(yr, t)

        del tape

        return yl_t ,yr_t

    def loss_fn(self):

        t_0 = tf.zeros((1,1))

        # reaidual(잔차)
        r = self.get_r()[2]
        phi_r = tf.reduce_mean(tf.square(r))

        loss = phi_r

        #Add phi^0 and phi^b to the loss
        # 초기 조건(yr,yl의 좌표)
        u_0 = self.model(t_0)
        init_a = tf.reshape(u_0[:,0],shape=(-1,1))
        init_b = tf.reshape(u_0[:,1],shape=(-1,1))
        init_R = tf.reshape(u_0[:,2],shape=(-1,1))
        init_theta_l =tf.reshape(u_0[:,3],shape=(-1,1))
        init_theta_r = tf.reshape(u_0[:,4],shape=(-1,1))

        init_yl = init_R*tf.sin(init_theta_l)+init_b
        init_yr = init_R*tf.sin(init_theta_r)+init_b

        loss += tf.reduce_mean(tf.square(yl_0 - init_yl))
        loss += tf.reduce_mean(tf.square(yr_0 - init_yr))


        # 경계 조건(속도)-1
        for i in range(len(t_data)):
            u = self.model(t_data[i])
            #pred_a = tf.reshape(u[:,0],shape=(-1,1))
            pred_b = tf.reshape(u[:,1],shape=(-1,1))
            pred_R = tf.reshape(u[:,2],shape=(-1,1))
            pred_theta_l =tf.reshape(u[:,3],shape=(-1,1))
            pred_theta_r = tf.reshape(u[:,4],shape=(-1,1))

            pred_yl = pred_R*tf.sin(pred_theta_l)+pred_b
            pred_yr = pred_R*tf.sin(pred_theta_r)+pred_b

            loss += tf.reduce_mean(tf.square(pred_yl - yl_0*vl*t_data[i][0]))
            loss += tf.reduce_mean(tf.square(pred_yr - yr_0*vr*t_data[i][0]))

        # 경계 조건(속도)-2
        # yl_t ,yr_t = self.get_boundary()
        # loss = yr_t+yr_t

        return loss


    def get_grad(self):
        with tf.GradientTape(persistent=True) as tape:
            # This tape is for derivatives with
            # respect to trainable variables
            tape.watch(self.model.trainable_variables)
            loss = self.loss_fn()

        g = tape.gradient(loss, self.model.trainable_variables)
        del tape

        return loss, g


    def solve_with_TFoptimizer(self, optimizer, N=1001):
        """This method performs a gradient descent type optimization."""

        @tf.function
        def train_step():
            loss, grad_theta = self.get_grad()

            # Perform gradient descent step
            optimizer.apply_gradients(zip(grad_theta, self.model.trainable_variables))
            return loss

        for i in range(N):

            loss = train_step()

            self.current_loss = loss.numpy()
            self.callback()

    def solve_with_ScipyOptimizer(self, method='L-BFGS-B', **kwargs):
        """This method provides an interface to solve the learning problem
        using a routine from scipy.optimize.minimize.
        (Tensorflow 1.xx had an interface implemented, which is not longer
        supported in Tensorflow 2.xx.)
        Type conversion is necessary since scipy-routines are written in Fortran
        which requires 64-bit floats instead of 32-bit floats."""

        def get_weight_tensor():
            """Function to return current variables of the model
            as 1d tensor as well as corresponding shapes as lists."""

            weight_list = []
            shape_list = []

            # Loop over all variables, i.e. weight matrices, bias vectors and unknown parameters
            for v in self.model.variables:
                shape_list.append(v.shape)
                weight_list.extend(v.numpy().flatten())

            weight_list = tf.convert_to_tensor(weight_list)
            return weight_list, shape_list

        x0, shape_list = get_weight_tensor()

        def set_weight_tensor(weight_list):
            """Function which sets list of weights
            to variables in the model."""
            idx = 0
            for v in self.model.variables:
                vs = v.shape

                # Weight matrices
                if len(vs) == 2:
                    sw = vs[0]*vs[1]
                    new_val = tf.reshape(weight_list[idx:idx+sw],(vs[0],vs[1]))
                    idx += sw

                # Bias vectors
                elif len(vs) == 1:
                    new_val = weight_list[idx:idx+vs[0]]
                    idx += vs[0]

                # Variables (in case of parameter identification setting)
                elif len(vs) == 0:
                    new_val = weight_list[idx]
                    idx += 1

                # Assign variables (Casting necessary since scipy requires float64 type)
                v.assign(tf.cast(new_val, DTYPE))

        def get_loss_and_grad(w):
            """Function that provides current loss and gradient
            w.r.t the trainable variables as vector. This is mandatory
            for the LBFGS minimizer from scipy."""

            # Update weights in model
            set_weight_tensor(w)
            # Determine value of \phi and gradient w.r.t. \theta at w
            loss, grad = self.get_grad()

            # Store current loss for callback function
            loss = loss.numpy().astype(np.float64)
            self.current_loss = loss

            # Flatten gradient
            grad_flat = []
            for g in grad:
                grad_flat.extend(g.numpy().flatten())

            # Gradient list to array
            grad_flat = np.array(grad_flat,dtype=np.float64)

            # Return value and gradient of \phi as tuple
            return loss, grad_flat


        return scipy.optimize.minimize(fun=get_loss_and_grad,
                                       x0=x0,
                                       jac=True,
                                       method=method,
                                       callback=self.callback,
                                       **kwargs)

    def callback(self, xr=None):
        if self.iter % 50 == 0:
            print('It {:05d}: loss = {:10.8e}'.format(self.iter,self.current_loss))
        self.hist.append(self.current_loss)
        self.iter+=1

    def plot_loss_history(self, ax=None):
        if not ax:
            fig = plt.figure(figsize=(7,5))
            ax = fig.add_subplot(111)
        ax.semilogy(range(len(self.hist)), self.hist,'k-')
        ax.set_xlabel('$n_{epoch}$')
        ax.set_ylabel('$\\phi^{n_{epoch}}$')
        return ax

In [9]:
model = PINN_NeuralNet(t_data)

NameError: name 'PINN_NeuralNet' is not defined

In [ ]:
model.build(input_shape=(None,1))
solver = PINNSolver(model,t_data)

In [18]:
print(t_data)

tf.Tensor(
[[1.27272500e-06]
 [8.12988319e-06]
 [1.36279060e-05]
 ...
 [2.79928773e-02]
 [2.79933925e-02]
 [2.79937181e-02]], shape=(10000, 1), dtype=float64)


In [36]:
new_t_data = tf.Variable(tf.zeros_like((0,1),dtype=DTYPE))
print(new_t_data)
print(tf.constant(-0.0006874250504951296, shape=(1,1),dtype=tf.float64))
for i in range(len(t_data)) :
        print(tf.constant(t_data[i][0], shape=(1,1),dtype=tf.float64))

<tf.Variable 'Variable:0' shape=(2,) dtype=float64, numpy=array([0., 0.])>
tf.Tensor([[-0.00068743]], shape=(1, 1), dtype=float64)
tf.Tensor([[1.272725e-06]], shape=(1, 1), dtype=float64)
tf.Tensor([[8.12988319e-06]], shape=(1, 1), dtype=float64)
tf.Tensor([[1.3627906e-05]], shape=(1, 1), dtype=float64)
tf.Tensor([[1.60887269e-05]], shape=(1, 1), dtype=float64)
tf.Tensor([[1.64038655e-05]], shape=(1, 1), dtype=float64)
tf.Tensor([[1.70918272e-05]], shape=(1, 1), dtype=float64)
tf.Tensor([[1.86046233e-05]], shape=(1, 1), dtype=float64)
tf.Tensor([[1.91380948e-05]], shape=(1, 1), dtype=float64)
tf.Tensor([[2.20558462e-05]], shape=(1, 1), dtype=float64)
tf.Tensor([[2.66726654e-05]], shape=(1, 1), dtype=float64)
tf.Tensor([[2.7584078e-05]], shape=(1, 1), dtype=float64)
tf.Tensor([[2.8969929e-05]], shape=(1, 1), dtype=float64)
tf.Tensor([[3.40342003e-05]], shape=(1, 1), dtype=float64)
tf.Tensor([[4.35139355e-05]], shape=(1, 1), dtype=float64)
tf.Tensor([[5.00472088e-05]], shape=(1, 1), dtyp

In [14]:
# Initialize model
from time import time

model = PINN_NeuralNet(t_data)
model.build(input_shape=(None,1))

# Initilize PINN solver
solver = PINNSolver(model,t_data)

# Decide which optimizer should be used
#mode = 'TFoptimizer'
mode = 'ScipyOptimizer'

# Start timer
t0 = time()

if mode == 'TFoptimizer':
    # Choose optimizer
    lr = tf.keras.optimizers.schedules.PiecewiseConstantDecay([1000,3000],[1e-2,1e-3,5e-4])
    optim = tf.keras.optimizers.Adam(learning_rate=lr)
    solver.solve_with_TFoptimizer(optim, t_data, N=4001)

elif mode == 'ScipyOptimizer':
    solver.solve_with_ScipyOptimizer(
                            method='L-BFGS-B',
                            options={'maxiter': 50000,
                                     'maxfun': 50000,
                                     'maxcor': 50,
                                     'maxls': 50,
                                     'ftol': 1.0*np.finfo(float).eps})

# Print computation time
print('\nComputation time: {} seconds'.format(time()-t0))

ValueError: Exception encountered when calling layer "pinn__neural_net_2" "                 f"(type PINN_NeuralNet).

Input 0 of layer "dense_18" is incompatible with the layer: expected min_ndim=2, found ndim=0. Full shape received: ()

Call arguments received by layer "pinn__neural_net_2" "                 f"(type PINN_NeuralNet):
  • X=tf.Tensor(shape=(), dtype=float64)

In [12]:
print(t_data)

tf.Tensor(
[[1.27272500e-06]
 [8.12988319e-06]
 [1.36279060e-05]
 ...
 [2.79928773e-02]
 [2.79933925e-02]
 [2.79937181e-02]], shape=(10000, 1), dtype=float64)


In [ ]:
#solver.plot_solution();
solver.plot_loss_history();

In [ ]:
solver.plot_solution();
solver.plot_loss_history();

In [ ]:
 # def get_r(self):
    #
    #     # A tf.GradientTape is used to compute derivatives in TensorFlow
    #     with tf.GradientTape(persistent=True) as tape:
    #         # Split t and x to compute partial derivatives
    #         t = self.t_data
    #
    #         # Variables t and x are watched during tape
    #         tape.watch(t)
    #
    #         # Determine residual
    #         u = self.model(t_data)
    #         a = tf.reshape(u[:,0],shape=(-1,1))
    #         b = tf.reshape(u[:,1],shape=(-1,1))
    #         R = tf.reshape(u[:,2],shape=(-1,1))
    #         theta_l =tf.reshape(u[:,3],shape=(-1,1))
    #         theta_r = tf.reshape(u[:,4],shape=(-1,1))
    #
    #         yl = R*tf.sin(theta_l)+b
    #         yr = R*tf.sin(theta_r)+b
    #         area = cal_area(a,b,d,theta_l,theta_r,R)
    #
    #     yl_t = tape.gradient(yl, t)
    #     yr_t = tape.gradient(yr, t)
    #     area_t = tape.gradient(area, t)
    #
    #     del tape
    #
    #     return area_t

# initial_data = [a0 ,b0,r0 ,theta_l0,theta_r0 ]
# cal_area_theta(center_a,center_b,d,theta_l,theta_r,R):
#initial_area = tf.constant(5.375716695978319e-08,shape=(1,1) , dtype=tf.float64)
#print(initial_area)
#thetar_d = tf.constant(initial_area, dtype=tf.float64)
# (yl_t-vl) + (yr_t-vr)
# Define residual of the condition

    # def plot_solution(self, **kwargs):
    #     N = 600
    #     tspace = np.linspace(self.model.lb[0], self.model.ub[0], N+1)
    #     xspace = np.linspace(self.model.lb[1], self.model.ub[1], N+1)
    #     T, X = np.meshgrid(tspace, xspace)
    #     Xgrid = np.vstack([T.flatten(),X.flatten()]).T
    #     upred = self.model(tf.cast(Xgrid,DTYPE))
    #     U = upred.numpy().reshape(N+1,N+1)
    #     fig = plt.figure(figsize=(9,6))
    #     ax = fig.add_subplot(111, projection='3d')
    #     ax.plot_surface(T, X, U, cmap='viridis', **kwargs)
    #     ax.set_xlabel('$t$')
    #     ax.set_ylabel('$x$')
    #     ax.set_zlabel('$u_\\theta(t,x)$')
    #     ax.view_init(35,35)
    #     return ax
